# 머신러닝 모델 활용

In [3]:
import numpy as np
import pandas as pd 
import lightgbm as lgb

In [4]:
data = pd.read_csv('한국가스공사_시간별 공급량_20181231.csv', encoding ='cp949')

In [7]:
data['연월일'] = pd.to_datetime(data['연월일'])

In [9]:
data['year'] = data['연월일'].dt.year
data['month'] = data['연월일'].dt.month
data['day'] = data['연월일'].dt.day
data['weekday'] = data['연월일'].dt.weekday

In [12]:
d_map ={}
for i, d in enumerate(data['구분'].unique()):
    d_map[d] = i 
data['구분'] = data['구분'].map(d_map)

In [18]:
data

,연월일,시간,구분,공급량,year,month,day,weekday
0,2013-01-01,1,0,2497.129,2013,1,1,1
1,2013-01-01,2,0,2363.265,2013,1,1,1
2,2013-01-01,3,0,2258.505,2013,1,1,1
3,2013-01-01,4,0,2243.969,2013,1,1,1
4,2013-01-01,5,0,2344.105,2013,1,1,1
...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0
368084,2018-12-31,21,6,669.961,2018,12,31,0
368085,2018-12-31,22,6,657.941,2018,12,31,0
368086,2018-12-31,23,6,610.953,2018,12,31,0


In [19]:
y = data['공급량']
x = data[['year','month','day','weekday','구분']]

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, shuffle = True, random_state = 1000)

In [26]:
d_train = lgb.Dataset(X_train, y_train)
d_test = lgb.Dataset(X_test, y_test)

In [27]:
params = {'objective' : 'regression',
         'metric':'mae',
         'seed' : 1000}

In [29]:
lgbr = lgb.train(params, d_train, 500, d_test, verbose_eval = 20, early_stopping_rounds = 100)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001669 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 66
[LightGBM] [Info] Number of data points in the train set: 276066, number of used features: 5
[LightGBM] [Info] Start training from score 947.304972
Training until validation scores don't improve for 100 rounds
[20]	valid_0's l1: 232.256
[40]	valid_0's l1: 192.599
[60]	valid_0's l1: 186.334
[80]	valid_0's l1: 183.958
[100]	valid_0's l1: 182.804
[120]	valid_0's l1: 181.77
[140]	valid_0's l1: 180.943
[160]	valid_0's l1: 180.059
[180]	valid_0's l1: 179.45
[200]	valid_0's l1: 178.282
[220]	valid_0's l1: 177.758
[240]	valid_0's l1: 177.37
[260]	valid_0's l1: 176.894
[280]	valid_0's l1: 176.562
[300]	valid_0's l1: 176.134
[320]	valid_0's l1: 175.754
[340]	valid_0's l1: 175.58
[360]	valid_0's l1: 175.266
[380]	valid_0's l1: 175.032
[400]

In [32]:
test = pd.read_csv('가스공급량 수요예측모델개발_data/test.csv')
sub = pd.read_csv('가스공급량 수요예측모델개발_data/sample_submission.csv')

In [37]:
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

In [40]:
test['일자'] = pd.to_datetime(test['일자'])

In [41]:
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday

In [44]:
test['구분'] = test['구분'].map(d_map)

In [47]:
test_x = test[['year','month','day','weekday','구분']]

In [49]:
pred = lgbr.predict(test_x)

In [51]:
sub['공급량']= pred

In [52]:
sub.to_csv('submission/baseline_lgbm.csv', index = False)